In [28]:
import numpy as np
import astropy.io.fits as fits
import astropy.units as u
from IPython.display import clear_output
import time

from scipy.ndimage import rotate
from scipy.interpolate import RegularGridInterpolator
import h5py
import os
from pathlib import Path
import glob

import misc
from matplotlib.patches import Circle
from importlib import reload
reload(misc)

data_dir = Path('/groups/douglase/kians-data-files/disk-processing/')

wavelength_c = 575e-9*u.m
D = 2.3631*u.m
mas_per_lamD = (wavelength_c/D*u.radian).to(u.mas)

# define desired PSF dimensions and pixelscale in units of lambda/D
npsf = 64
psf_pixelscale = 13e-6
psf_pixelscale_lamD = 500/575 * 1/2
psf_pixelscale_mas = psf_pixelscale_lamD*mas_per_lamD/u.pix

polaxis = 10

ndisk = 128
disk_pixelscale_mas = 10.5*u.mas/u.pix
disk_pixelscale_lamD = (disk_pixelscale_mas/mas_per_lamD).value

band1_wavelength = 575e-9*u.m
iwa_band1 = 2.8
owa_band1 = 9.7

iwa = iwa_band1 * wavelength_c/band1_wavelength
owa = owa_band1 * wavelength_c/band1_wavelength

iwa_mas = iwa*mas_per_lamD
owa_mas = owa*mas_per_lamD

disk_fpaths = sorted(glob.glob(str(data_dir/'disks'/'Debes_Models_072022'/'*')))
disk_fpaths

['/groups/douglase/kians-data-files/disk-processing/disks/Debes_Models_072022/annulus_inc0_r0.10.fits',
 '/groups/douglase/kians-data-files/disk-processing/disks/Debes_Models_072022/annulus_inc0_r0.15.fits',
 '/groups/douglase/kians-data-files/disk-processing/disks/Debes_Models_072022/annulus_inc0_r0.20.fits',
 '/groups/douglase/kians-data-files/disk-processing/disks/Debes_Models_072022/annulus_inc0_r0.30.fits',
 '/groups/douglase/kians-data-files/disk-processing/disks/Debes_Models_072022/annulus_inc0_r0.40.fits',
 '/groups/douglase/kians-data-files/disk-processing/disks/Debes_Models_072022/annulus_inc0_r0.50.fits',
 '/groups/douglase/kians-data-files/disk-processing/disks/Debes_Models_072022/annulus_inc0_r0.60.fits',
 '/groups/douglase/kians-data-files/disk-processing/disks/Debes_Models_072022/annulus_inc0_r0.80.fits',
 '/groups/douglase/kians-data-files/disk-processing/disks/Debes_Models_072022/annulus_inc30_r0.10.fits',
 '/groups/douglase/kians-data-files/disk-processing/disks/Debes

# Do the matrix multiplication

In [31]:
ipsf_fpath = data_dir/'ipsfs'/'hlc_band1_ipsfs_072022.hdf5'
f = h5py.File(ipsf_fpath, 'r')
ipsf_matrix = np.array(f['ipsf_matrix'])
f.close()

In [32]:
for i,fpath in enumerate(disk_fpaths):
    disk = misc.pad_or_crop(fits.getdata(fpath), ndisk)
    image = np.reshape(ipsf_matrix.dot(disk.flatten()), (npsf,npsf))
    
    hdr = fits.Header()
    hdr['PXSCLAMD'] = psf_pixelscale_lamD
    hdr.comments['PXSCLAMD'] = 'pixel scale in lam0/D per pixel'
    hdr['PXSCLMAS'] = psf_pixelscale_mas.value
    hdr.comments['PXSCLMAS'] = 'pixel scale in mas per pixel'
    hdr['PIXELSCL'] = psf_pixelscale
    hdr.comments['PIXELSCL'] = 'pixel scale in meters per pixel'
    hdr['CWAVELEN'] = wavelength_c.value
    hdr.comments['CWAVELEN'] = 'central wavelength in meters'
    hdr['BANDPASS'] = 0
    hdr.comments['BANDPASS'] = 'bandpass as fraction of CWAVELEN'
    
    image_hdu = fits.PrimaryHDU(data=image, header=hdr)
    image_hdu.writeto(fpath.replace('.fits', '_image.fits'), overwrite=True)
    print(i+1, fpath.replace('.fits', '_image.fits') )

1 /groups/douglase/kians-data-files/disk-processing/disks/Debes_Models_072022/annulus_inc0_r0.10_image.fits
2 /groups/douglase/kians-data-files/disk-processing/disks/Debes_Models_072022/annulus_inc0_r0.15_image.fits
3 /groups/douglase/kians-data-files/disk-processing/disks/Debes_Models_072022/annulus_inc0_r0.20_image.fits
4 /groups/douglase/kians-data-files/disk-processing/disks/Debes_Models_072022/annulus_inc0_r0.30_image.fits
5 /groups/douglase/kians-data-files/disk-processing/disks/Debes_Models_072022/annulus_inc0_r0.40_image.fits
6 /groups/douglase/kians-data-files/disk-processing/disks/Debes_Models_072022/annulus_inc0_r0.50_image.fits
7 /groups/douglase/kians-data-files/disk-processing/disks/Debes_Models_072022/annulus_inc0_r0.60_image.fits
8 /groups/douglase/kians-data-files/disk-processing/disks/Debes_Models_072022/annulus_inc0_r0.80_image.fits
9 /groups/douglase/kians-data-files/disk-processing/disks/Debes_Models_072022/annulus_inc30_r0.10_image.fits
10 /groups/douglase/kians-d

82 /groups/douglase/kians-data-files/disk-processing/disks/Debes_Models_072022/ring_inc89_r0.15_image.fits
83 /groups/douglase/kians-data-files/disk-processing/disks/Debes_Models_072022/ring_inc89_r0.20_image.fits
84 /groups/douglase/kians-data-files/disk-processing/disks/Debes_Models_072022/ring_inc89_r0.30_image.fits
85 /groups/douglase/kians-data-files/disk-processing/disks/Debes_Models_072022/ring_inc89_r0.40_image.fits
86 /groups/douglase/kians-data-files/disk-processing/disks/Debes_Models_072022/ring_inc89_r0.50_image.fits
87 /groups/douglase/kians-data-files/disk-processing/disks/Debes_Models_072022/ring_inc89_r0.60_image.fits
88 /groups/douglase/kians-data-files/disk-processing/disks/Debes_Models_072022/ring_inc89_r0.80_image.fits
89 /groups/douglase/kians-data-files/disk-processing/disks/Debes_Models_072022/ring_inc89_r1.00_image.fits
90 /groups/douglase/kians-data-files/disk-processing/disks/Debes_Models_072022/ring_inc89_r1.20_image.fits
91 /groups/douglase/kians-data-files/